In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
!ls /content/drive/MyDrive/Alzheimer_s\ Dataset.zip

'/content/drive/MyDrive/Alzheimer_s Dataset.zip'


In [ ]:
#try 1 train part
#( 10 epochs ,test accuracy : 0.67)

In [5]:
import os
import zipfile
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Define the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/Alzheimer_s Dataset.zip"

extracted_dir = "/content/Alzheimer_s_Dataset"  # Define the directory where files will be extracted

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Get the path to the inner directory containing the dataset files
inner_dir = os.path.join(extracted_dir, 'Alzheimer_s Dataset')

# Define paths to train data directory
train_dir = os.path.join(inner_dir, 'train')

# Check if the train directory exists
if not os.path.exists(train_dir):
    print("Train directory not found:", train_dir)
    exit()

# Define image size and batch size
img_size = (224, 224)
batch_size = 30

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # Assuming images are organized in folders by class
)

# Load MobileNet model without top (classification) layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),

    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10)

# Save the trained model
model.save("trained_model.h5")


Found 5121 images belonging to 4 classes.
17225924/17225924 [==============================] - 0s 0us/step
Epoch 1/10
171/171 [==============================] - 1036s 6s/step - loss: 0.9061 - accuracy: 0.6112
Epoch 2/10
171/171 [==============================] - 997s 6s/step - loss: 0.6217 - accuracy: 0.7329
Epoch 3/10
171/171 [==============================] - 991s 6s/step - loss: 0.4873 - accuracy: 0.7952
Epoch 4/10
171/171 [==============================] - 989s 6s/step - loss: 0.3861 - accuracy: 0.8481
Epoch 5/10
171/171 [==============================] - 991s 6s/step - loss: 0.3172 - accuracy: 0.8774
Epoch 6/10
171/171 [==============================] - 991s 6s/step - loss: 0.2764 - accuracy: 0.8922
Epoch 7/10
171/171 [==============================] - 993s 6s/step - loss: 0.2147 - accuracy: 0.9184
Epoch 8/10
171/171 [==============================] - 999s 6s/step - loss: 0.2051 - accuracy: 0.9242
Epoch 9/10
171/171 [==============================] - 999s 6s/step - loss: 0.1654 - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#try 1 test part (test accuracy : 0.67)

In [9]:
# Define paths to test data directory
test_dir = os.path.join(inner_dir, 'test')

# Check if the test directory exists
if not os.path.exists(test_dir):
    print("Test directory not found:", test_dir)
    exit()

# Load test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming images are organized in folders by class
    shuffle=False
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

# Predict labels for test data
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = test_generator.classes

# Calculate performance metrics
from sklearn.metrics import classification_report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices)
print("Classification Report:")
print(report)


Found 1279 images belonging to 4 classes.
43/43 [==============================] - 59s 1s/step - loss: 2.0263 - accuracy: 0.6701
Test Accuracy: 0.6700547337532043
43/43 [==============================] - 50s 1s/step
Classification Report:
                  precision    recall  f1-score   support

    MildDemented       1.00      0.01      0.02       179
ModerateDemented       0.00      0.00      0.00        12
     NonDemented       0.63      0.96      0.76       640
VeryMildDemented       0.80      0.54      0.64       448

        accuracy                           0.67      1279
       macro avg       0.61      0.38      0.36      1279
    weighted avg       0.73      0.67      0.61      1279



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
model.save('/content/drive/MyDrive/model_1.h5')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf

# Path to your model
model_path = '/content/drive/MyDrive/model_1.h5'

# Load the model
model = tf.keras.models.load_model(model_path)

In [ ]:
#try 2 train part
#( 50 epochs ,test accuracy : 0.69)

In [5]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Define the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/Alzheimer_s Dataset.zip"

extracted_dir = "/content/Alzheimer_s_Dataset"  # Define the directory where files will be extracted

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Get the path to the inner directory containing the dataset files
inner_dir = os.path.join(extracted_dir, 'Alzheimer_s Dataset')

# Define paths to train data directory
train_dir = os.path.join(inner_dir, 'train')

# Check if the train directory exists
if not os.path.exists(train_dir):
    print("Train directory not found:", train_dir)
    exit()

# Define image size and batch size
img_size = (224, 224)
batch_size = 30

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Additional rotation
    width_shift_range=0.2,  # Additional horizontal shift
    height_shift_range=0.2,  # Additional vertical shift
    fill_mode='nearest'  # Fill in missing pixels with the nearest available value
)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # Assuming images are organized in folders by class
)

# Load MobileNet model without top (classification) layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers with regularization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Dropout for regularization
    Dense(512, activation='relu'),  # Reduced number of neurons
    Dropout(0.5),  # Dropout for regularization
    Dense(256, activation='relu'),  # Further reduced number of neurons
    BatchNormalization(),  # Batch normalization layer
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_generator, epochs=50, callbacks=[early_stopping])

# Save the trained model
model.save("trained_model.h5")

Found 5121 images belonging to 4 classes.
Epoch 1/50
171/171 [==============================] - 89s 384ms/step - loss: 1.0234 - accuracy: 0.5667
Epoch 2/50
171/171 [==============================] - 67s 392ms/step - loss: 0.7440 - accuracy: 0.6639
Epoch 3/50
171/171 [==============================] - 67s 391ms/step - loss: 0.6403 - accuracy: 0.7237
Epoch 4/50
171/171 [==============================] - 67s 391ms/step - loss: 0.5294 - accuracy: 0.7838
Epoch 5/50
171/171 [==============================] - 67s 388ms/step - loss: 0.4652 - accuracy: 0.8133
Epoch 6/50
171/171 [==============================] - 67s 392ms/step - loss: 0.3689 - accuracy: 0.8535
Epoch 7/50
171/171 [==============================] - 69s 401ms/step - loss: 0.3348 - accuracy: 0.8723
Epoch 8/50
171/171 [==============================] - 66s 384ms/step - loss: 0.2965 - accuracy: 0.8901
Epoch 9/50
171/171 [==============================] - 67s 392ms/step - loss: 0.2621 - accuracy: 0.9043
Epoch 10/50
171/171 [==========

In [7]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
model.save("/content/drive/MyDrive/trained_model2.h5")

In [ ]:
#try 2 test part
#( test accuracy : 0.69)

In [12]:
# Define paths to test data directory
test_dir = os.path.join(inner_dir, 'test')

# Check if the test directory exists
if not os.path.exists(test_dir):
    print("Test directory not found:", test_dir)
    exit()

# Load test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming images are organized in folders by class
    shuffle=False
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

# Predict labels for test data
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = test_generator.classes

# Calculate performance metrics
from sklearn.metrics import classification_report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices)
print("Classification Report:")
print(report)


Found 1279 images belonging to 4 classes.
43/43 [==============================] - 3s 58ms/step - loss: 1.6936 - accuracy: 0.6927
Test Accuracy: 0.6927286982536316
43/43 [==============================] - 2s 43ms/step
Classification Report:
                  precision    recall  f1-score   support

    MildDemented       0.74      0.31      0.44       179
ModerateDemented       0.88      0.58      0.70        12
     NonDemented       0.90      0.66      0.76       640
VeryMildDemented       0.55      0.89      0.68       448

        accuracy                           0.69      1279
       macro avg       0.77      0.61      0.65      1279
    weighted avg       0.76      0.69      0.69      1279

